# pivot
- 데이터프레임에서 두개의 열을 이용하여 행/열 인덱스 reshape된 테이블을 통한 집계
- groupby와 통계함수를 한번에 적용할 수 있다! 
- 새로운 데이터프레임이 만들어진다
- pivot(index, columns, data.value)
- pivot_table(data, values, index, cloumns, aggfunc=) 
- ex) aggfunc=[np.sum, np.mean]처럼 함수를 여러개 쓸수도 있다.

In [2]:
# 필요한 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import warnings
warnings.filterwarnings(action = 'ignore') 

from datetime import date, datetime, timedelta
from dateutil.parser import parse

print("numpy version:",np.__version__) 
print("pandas version:",pd.__version__)

numpy version: 1.20.1
pandas version: 1.2.4


In [3]:
data = {
    'city'   : ['seoul','seoul','seoul','busan','busan','busan','kwangju','kwangju'],
    'year'   : ['2015','2010','2005','2015','2010','2005','2015','2010'],
    'pop'    : [100,200,300,400,500,600,700,800],
    'region' : ['수도권','수도권','수도권','경상권','경상권','경상권','전라권','전라권']
}
pivot_frm=pd.DataFrame(data)
pivot_frm

,city,year,pop,region
0,seoul,2015,100,수도권
1,seoul,2010,200,수도권
2,seoul,2005,300,수도권
3,busan,2015,400,경상권
4,busan,2010,500,경상권
5,busan,2005,600,경상권
6,kwangju,2015,700,전라권
7,kwangju,2010,800,전라권


In [5]:
pivot_frm.pivot(['region','city'],'year','pop') # 다중인덱스도 가능! 

,year,2005,2010,2015
region,city,,,
경상권,busan,600.0,500.0,400.0
수도권,seoul,300.0,200.0,100.0
전라권,kwangju,NaN,800.0,700.0


In [35]:
import seaborn as sns
iris_datasets= sns.load_dataset('iris')
titanic_datasets = sns.load_dataset('titanic')

In [36]:
titanic_datasets.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [37]:
pclass_sex_frm = titanic_datasets.loc[:,['pclass','sex']]
pclass_sex_frm


,pclass,sex
0,3,male
1,1,female
2,3,female
3,1,female
4,3,male
...,...,...
886,2,male
887,1,female
888,3,female
889,1,male


In [38]:
# 성별과 객실등급에 따른 승객 수 집계한다면?
pclass_sex_pivot_frm = pclass_sex_frm.groupby(['sex','pclass']).size().reset_index(name='cnt')
pclass_sex_pivot_frm
pclass_sex_pivot_frm.pivot('sex','pclass','cnt')

pclass,1,2,3
sex,,,
female,94,76,144
male,122,108,347


In [41]:
# 성별과 생존여부에 따른 승객수 집계한다면?
sex_survivied_frm=titanic_datasets[['sex','survived']]
sex_survivied_pivot_frm = sex_survivied_frm.groupby(['sex','survived']).size().reset_index(name='cnt')
sex_survivied_pivot_frm.pivot('sex','survived','cnt')

survived,0,1
sex,,
female,81,233
male,468,109


In [46]:
titanic_datasets['cnt']=1
titanic_datasets.pivot_table(index='sex',columns='survived',values='cnt',
                             aggfunc=np.sum,margins=True, margins_name='분석결과')

survived,0,1,분석결과
sex,,,
female,81,233,314
male,468,109,577
분석결과,549,342,891


In [47]:
pivot_frm

,city,year,pop,region
0,seoul,2015,100,수도권
1,seoul,2010,200,수도권
2,seoul,2005,300,수도권
3,busan,2015,400,경상권
4,busan,2010,500,경상권
5,busan,2005,600,경상권
6,kwangju,2015,700,전라권
7,kwangju,2010,800,전라권


In [48]:
pivot_frm.pivot_table(index='city',columns='year',values='pop', 
                             margins=True, margins_name='분석결과')

year,2005,2010,2015,분석결과
city,,,,
busan,600.0,500.0,400.0,500
kwangju,NaN,800.0,700.0,750
seoul,300.0,200.0,100.0,200
분석결과,450.0,500.0,400.0,450


In [49]:
tips_datasets = sns.load_dataset('tips')
tips_datasets

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [52]:
tips_datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [72]:
# 식사대금 대비 팁의 비율이 가장 높은 데이터를 찾는다면?
tips_datasets['tip_ratio']=tips_datasets['tip']/tips_datasets['total_bill'] # 비율값을 가지는 칼럼을 새롭게 추가
# 방법1
print(tips_datasets[tips_datasets['tip_ratio']==np.max(tips_datasets['tip_ratio'])])
# 방법2
print(tips_datasets.iloc[tips_datasets['tip_ratio'].idxmax()])


     total_bill   tip   sex smoker  day    time  size  total/tip  tip_ratio
172        7.25  5.15  Male    Yes  Sun  Dinner     2   0.710345   0.710345
total_bill        7.25
tip               5.15
sex               Male
smoker             Yes
day                Sun
time            Dinner
size                 2
total/tip     0.710345
tip_ratio     0.710345
Name: 172, dtype: object


In [77]:
# 성별에 따른 집계
tips_datasets.groupby('sex').count()

,total_bill,tip,smoker,day,time,size,total/tip,tip_ratio
sex,,,,,,,,
Male,157,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87,87


In [74]:
tips_datasets.groupby('sex').size()

sex
Male      157
Female     87
dtype: int64

In [79]:
# 성별과 흡연 유무에 따른 팁 비율?
tips_datasets.groupby(['sex','smoker']).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [81]:
tips_datasets.pivot_table(index='sex',columns='smoker',values='tip_ratio', aggfunc='count',margins=True )
# aggfunc='함수이름' 이런식으로 적어줘도 OK! 

smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


#### qcut() : 범주를 만든다 (동일한 개수) - 그룹에 대한 통계량을 확인 가능
#### cut() : 범주를 만든다 (동일한 길이) -   그룹에 대한 통계량을 확인 가능

In [86]:
# 각 붓꽃 꽃잎의 길이가 해당 종내에서 대/중/소 어느그룹에 해당되는지 알고 싶다면?
iris_grp=iris_datasets.groupby('species')
for key, group in iris_grp:
    print('key:',key)
    print('len:',len(group))
    print(group)
    print('*'*50)

key: setosa
len: 50
    sepal_length  sepal_width  petal_length  petal_width species
0            5.1          3.5           1.4          0.2  setosa
1            4.9          3.0           1.4          0.2  setosa
2            4.7          3.2           1.3          0.2  setosa
3            4.6          3.1           1.5          0.2  setosa
4            5.0          3.6           1.4          0.2  setosa
5            5.4          3.9           1.7          0.4  setosa
6            4.6          3.4           1.4          0.3  setosa
7            5.0          3.4           1.5          0.2  setosa
8            4.4          2.9           1.4          0.2  setosa
9            4.9          3.1           1.5          0.1  setosa
10           5.4          3.7           1.5          0.2  setosa
11           4.8          3.4           1.6          0.2  setosa
12           4.8          3.0           1.4          0.1  setosa
13           4.3          3.0           1.1          0.1  setosa
14   

In [87]:
iris_datasets.groupby('species').petal_length.size()

species
setosa        50
versicolor    50
virginica     50
Name: petal_length, dtype: int64

In [88]:
def cut_petal_length(s):
    return pd.cut(s, 3, labels=['대','중','소'])

In [89]:
iris_datasets['category'] = iris_datasets.groupby('species').petal_length.transform(cut_petal_length) 
# transform대신 apply도 사용 가능! 

In [90]:
iris_datasets

,sepal_length,sepal_width,petal_length,petal_width,species,category
0,5.1,3.5,1.4,0.2,setosa,중
1,4.9,3.0,1.4,0.2,setosa,중
2,4.7,3.2,1.3,0.2,setosa,대
3,4.6,3.1,1.5,0.2,setosa,중
4,5.0,3.6,1.4,0.2,setosa,중
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,대
146,6.3,2.5,5.0,1.9,virginica,대
147,6.5,3.0,5.2,2.0,virginica,대
148,6.2,3.4,5.4,2.3,virginica,중


In [99]:
# 종별로 꽃잎길이, 꽃잎폭의 평균을 구한다면?
iris_datasets.groupby('species').agg(np.mean).loc[:,['petal_length','petal_width']]

,petal_length,petal_width
species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [102]:
# 종별로 최대/최소 값을 구한다면?
iris_datasets.groupby('species').agg([np.min,np.max])

sepal_length      sepal_width      petal_length      petal_width  \
                   amin amax        amin amax         amin amax        amin   
species                                                                       
setosa              4.3  5.8         2.3  4.4          1.0  1.9         0.1   
versicolor          4.9  7.0         2.0  3.4          3.0  5.1         1.0   
virginica           4.9  7.9         2.2  3.8          4.5  6.9         1.4   

                category       
           amax     amin amax  
species                        
setosa      0.6        대    소  
versicolor  1.8        대    소  
virginica   2.5        대    소

In [129]:
titanic_datasets

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,cnt,age_category
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,1,B
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,1,C
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,1,B
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,1,C
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,1,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,1,B
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,1,A
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,1,NaN
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,1,B


In [136]:
# 성별, 선실, 나이그룹(age_qcut) 의한 생존율을 데이터프레임으로 생성하고
# 행에는 성별 및 나이그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다

ex_frm.groupby(['sex','pclass','age_category']).agg(np.mean)['survived'].values

array([0.92857143, 0.93333333, 1.        , 0.96875   , 1.        ,
       0.86363636, 0.95      , 0.875     , 0.5106383 , 0.53846154,
       0.47058824, 0.08333333, 0.57142857, 0.5625    , 0.5       ,
       0.27777778, 0.52631579, 0.        , 0.09677419, 0.08333333,
       0.19736842, 0.12658228, 0.16949153, 0.07692308])

In [162]:
print('[문제01] - ')
print('qcut 명령으로 세 개의 나이 그룹을 만든다. - ')
titanic_datasets = sns.load_dataset('titanic')
titanic_datasets['age_category'] = pd.qcut(titanic_datasets['age'] , 3 , labels = ['A' , 'B' , 'C' ])
titanic_datasets


[문제01] - 
qcut 명령으로 세 개의 나이 그룹을 만든다. - 


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_category
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,A
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,C
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,B
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,C
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,B
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,A
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,NaN
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,B


In [161]:
print()
print('[문제02] - ')
print('성별, 선실, 나이그룹(age_category) 의한 생존율을 데이터프레임으로 생성하고')
print('행에는 성별 및 나이그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다 -')
print('생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다. -')
print('cut - pivot_table')
# 방법 1 (null값 무시한 케이스)
ex_frm =titanic_datasets[['sex','pclass','age_category','survived']] # titanic에서 필요한 칼럼만 추출해서 새로운 dataframe todtjd
ex_frm =ex_frm.groupby(['sex','pclass','age_category']).agg(np.mean)

ex_frm_pivot = titanic_datasets.pivot_table(index=['sex','age_category'],columns='pclass',values='survived', margins=True )
ex_frm_pivot


# 방법 2 (전체인원수에서 생존률을 구한 것)
ex_frm1 = titanic_datasets.pivot_table(index=['sex','age_category'],
                                       columns='pclass',values='survived',
                                       margins=True, margins_name='survived_ratio',
                                      aggfunc='count')
display(ex_frm1)
survived_frm=frm/frm.loc['survived_ratio','survived_ratio'][0] # Series로 가져오기 위해서! 
display(survived_frm)


[문제02] - 
성별, 선실, 나이그룹(age_category) 의한 생존율을 데이터프레임으로 생성하고
행에는 성별 및 나이그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다 -
생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다. -
cut - pivot_table


pclass                      1         2         3       All
sex    age_category                                        
female A             0.954545  1.000000  0.508475  0.702970
       B             0.947368  0.909091  0.481481  0.772152
       C             0.977273  0.857143  0.250000  0.802469
male   A             0.500000  0.357143  0.158879  0.220690
       B             0.500000  0.076923  0.195652  0.209150
       C             0.347826  0.062500  0.055556  0.187097
All                  0.655914  0.479769  0.239437  0.406162

In [165]:
print('문제[03] - ')
print('성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다. - ')
ex_frm2=titanic_datasets.pivot_table(index='sex',
                                       columns='pclass',values='survived',
                                       margins=True, margins_name='survived_ratio',
                                      aggfunc='count')
display(ex_frm2)
survived_frm2=ex_frm2/titanic_datasets.shape[0]
display(survived_frm2)

문제[03] - 
성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다. - 


pclass,1,2,3,survived_ratio
sex,,,,
female,94,76,144,314
male,122,108,347,577
survived_ratio,216,184,491,891


pclass,1,2,3,survived_ratio
sex,,,,
female,0.105499,0.085297,0.161616,0.352413
male,0.136925,0.121212,0.389450,0.647587
survived_ratio,0.242424,0.206510,0.551066,1.000000
